In [2]:
import keras
keras.__version__

'2.1.5'

### convnet 개요
- convnet의 개념을 설명하고 이를 이용하여 MNIST 분류 문제를 다룬다
- 앞의 예에서는 97.8%의 성능을 보였다.

- 아래의 코드에서 Conv2D와 MaxPoolinf2D 두 계층을 사용하여 신경망을 구성한다
- 입력         ㅜㅡ텐서로 (image_height, image_width, image_channels)구조를 사용한다 (배치의 크기는 별도로 지정한다)
- 여기서는 (28, 28, 1)의 크기를 갖으며 이를 input_shape인자로 사용하면 된다

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

- 위에서 정의한 covnet의 구조를 보겠다

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


- conv2d와 MaxPooling 계층의 출력이 3차원 텐서로서 구조가 (height, width, channels)로 된다
- 폭과 높의 값은 네트워크가 깊어지면서 점차 작아진다.
- 채널 크기는 conv2d 계층을 만들 때 정해준다 (위에서 32나 64)
- 출력단의 (3, 3, 64) 구조의 출력 텐서를 이제 Dense 구조의 최종 분류 네트워크로 보내야 한다.
- 분류기는 벡터, 즉 1차원 텐서를 사용하므로, 현재 가지고 있는 3D 텐서를 먼저 1차원으로 만들어야 한다(flatten)

In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

- 10개 숫자를 구분해야 하므로 10개의 출력을 갖는 softmax 활성화 함수를 사용한다

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

- (3, 3, 64) 구조의 출력을 (576,) 구조로 평탄화 한 후 두개의 Dense 계층을 통과시켰다.
- 이제 훈련을 시키면 된다.

In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 24s 402us/step - loss: 0.1749 - acc: 0.9452
Epoch 2/5
60000/60000 [==============================] - 25s 419us/step - loss: 0.0512 - acc: 0.9839
Epoch 3/5
60000/60000 [==============================] - 25s 423us/step - loss: 0.0354 - acc: 0.9891
Epoch 4/5
60000/60000 [==============================] - 26s 438us/step - loss: 0.0280 - acc: 0.9919
Epoch 5/5
60000/60000 [==============================] - 26s 431us/step - loss: 0.0221 - acc: 0.9933


- 이제 테스트 데이터로 모델의 성능을 검증한다

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 137us/step


In [10]:
test_acc

0.9909

- 앞에서 소개한 '일반' 구조의 신경망 모델에 비해 convnet은 성능이 97.8%에서 99.1%로 개선된 것을 알 수 있다.